# Forest Clase

## Importar librerias y cargar conjunto de datos

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

alerta = pd.read_csv('conjunto de datos/ALERTA.csv', sep=';')
alerta = alerta.select_dtypes(include='number')
alerta = alerta.drop('UDI',axis=1)
alerta

,Temperatura_Aire,Temperatura_Proceso,Velocidad_Rotacion,Torque,Desgaste_Herramientas,Objetivo
0,298.1,308.6,1551,42.8,0,0
1,298.2,308.7,1408,46.3,3,0
2,298.1,308.5,1498,49.4,5,0
3,298.2,308.6,1433,39.5,7,0
4,298.2,308.7,1408,40.0,9,0
...,...,...,...,...,...,...
9995,298.8,308.4,1604,29.5,14,0
9996,298.9,308.4,1632,31.8,17,0
9997,299.0,308.6,1645,33.4,22,0
9998,299.0,308.7,1408,48.5,25,0


## Generar datos aleatorios en base a un porcentaje de la muestra inicial

In [17]:
print(alerta.sample(frac=2/3, replace=True)) #se toma aleatorio aproximadamente el 66% de los datos

      Temperatura_Aire  Temperatura_Proceso  Velocidad_Rotacion  Torque  \
1635             297.8                307.8                1383    56.5   
1600             297.8                307.8                1543    43.7   
5994             300.6                310.4                1608    39.2   
6198             301.0                310.9                1554    32.7   
3556             301.9                310.8                1735    31.5   
...                ...                  ...                 ...     ...   
4129             301.8                310.3                1666    32.7   
2025             298.7                308.5                1469    39.9   
5155             304.2                313.3                1625    37.3   
8843             297.3                308.8                1564    38.0   
7498             300.2                311.8                1748    28.5   

      Desgaste_Herramientas  Objetivo  
1635                     98         0  
1600               

## Seleccionar Caracteristicas (x) y clase (y)

### Clases de Objetivo
- 0 = No Fallas
- 1 = Tiene Fallas

In [18]:
x = alerta[alerta.columns[:-1]].values
x

y = alerta['Objetivo'].values
y

array([0, 0, 0, ..., 0, 0, 0], shape=(10000,))

### Generar bosque aleatorio

- **n_estimators:** `100` Son la cantidad de arboles a generar
- **Criterion:** `gini` Criterio de pureza o incertidumbre. Por que son muchos datos
- **max_features:** `sqrt` Selección de caracteristicas. Seria la raiz cuadrada de las caracteristicas. Se usa para que balancee las características. Va a calcular la raiz cuadrada de todas las caracteristicas en base al total de caracteristicas
- **boostrap:** `True` Muestreo con reemplazo. Seleccionar aleatoriamente los datos para entrenar el modelo y por ende los árboles de decisión con muestreo con reemplazo
- **max_samples:** `2/3` Muestra aleatoria para cada arbol. Fracción del muestreo en base a los datos originales
- **oob_score:** `True` Muestra para validación del bosque. Evaluar cada uno de los datos aleatorios con el porcentaje restande del muestreo original

In [21]:
bosque = RandomForestClassifier(n_estimators=200, criterion='gini', max_features='sqrt', bootstrap=True, max_samples=2/3, oob_score=True)

### Entrenamos al modelo

In [22]:
bosque.fit(x, y)

RandomForestClassifier(max_samples=0.6666666666666666, n_estimators=200,
                       oob_score=True)

### Predecimos un valor

In [23]:
print(bosque.predict([[16,0,1,40,10]]))
print(bosque.score(x,y)) #% clasificación correcta
print(bosque.oob_score_) #Puntaje de clasificación correcta de datos de prueba (1/3)

[0]
0.9979
0.9837


### Matriz de confusión

In [26]:
alerta2 = alerta.sample(frac=2/3, replace=True)

x_test = alerta2[alerta2.columns[:-1]].values
y_test = alerta2['Objetivo'].values

y_pred = bosque.predict(x_test)

mc = confusion_matrix(y_test, y_pred)
mc

array([[6443,    0],
       [  15,  209]])